In [ ]:
import pandas as pd
import re

In [ ]:
fields = ['username', 'text', 'date', 'label']

df = pd.DataFrame(columns=['Source', 'Target', 'Weight'])

extracted_tweets = []

list_of_topics = ['ukraine','covid','costofliving','brexit']

lbl = list_of_topics.index(topic)+1

filename = 'Aggregate_network_'+topic+'.csv'

In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [ ]:
idx = 0

df_csv_mp = pd.DataFrame(pd.read_csv('Labelled_tweets/mp_tweets_labelled.csv', skipinitialspace=True, usecols=fields))

mp_tweets = df_csv_mp.loc[(df_csv_mp['label'] == lbl)]

df_csv_journo = pd.DataFrame(pd.read_csv('Labelled_tweets/journalist_tweets_labelled.csv', skipinitialspace=True, usecols=fields))

journalist_tweets = df_csv_journo.loc[(df_csv_journo['label'] == lbl)]

tweet_data = mp_tweets[mp_tweets['username'].notna()].append\
    (journalist_tweets[journalist_tweets['username'].notna()])

In [ ]:
mp_list = list(pd.read_csv("User_list/MPs.csv")["Screen name"].dropna().str.replace('@',''))

journalist_df = pd.DataFrame(pd.read_csv("User_list/journalists.csv", usecols=['Handle'])).dropna()

journalist_list = list(journalist_df['Handle'].str.replace('@',''))

user_list = mp_list + journalist_list

In [ ]:
for index, row in tweet_data.iterrows():
    tweet = row['text']

    target = row['username']

    if '@' in tweet:

        result = re.findall(r"@(\w+)", tweet)

        retweeted_users = intersection(result, user_list)

        itr = 0

        while itr < len(retweeted_users):

            extracted_tweets.append(tweet)

            pos = df.index[(df['Source'] == retweeted_users[itr]) & (df['Target'] == target)].tolist()

            if len(pos) > 0:

                wt = df.at[pos[0], 'Weight']
                df.at[pos[0], 'Weight'] = wt+1

            else:

                df.loc[idx] = [retweeted_users[itr], target, 1]
                idx += 1

            itr += 1


df.to_csv(filename, index = False)